In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pickle

In [0]:
 with open('drive/My Drive/Machine_Learning-prj/vietnamese_language_model/tokenizer_s2_1M.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

In [0]:
with open('drive/My Drive/Machine_Learning-prj/vietnamese_language_model/sequences_digit_s2_1M.pkl', 'rb') as f:
    sequences_digit = pickle.load(f)

In [0]:
vocab_size = len(tokenizer.word_index) + 1

In [0]:
import keras

In [37]:
from keras.models import load_model

print("Loading pre-trained weight")
model = load_model('drive/My Drive/Machine_Learning-prj/vietnamese_language_model/51_weight_language_model.h5')

Loading pre-trained weight


In [8]:
!pip install pyvi

    100% |████████████████████████████████| 5.3MB 4.3MB/s 
    100% |████████████████████████████████| 757kB 15.0MB/s 
    100% |████████████████████████████████| 51kB 18.3MB/s 
  Running setup.py bdist_wheel for pyvi ... - \ | / - \ done
  Stored in directory: /root/.cache/pip/wheels/b8/cb/53/54834bcc54e39de0e83897064a70335902e15ac84b44125b1a
  Running setup.py bdist_wheel for tabulate ... - done
  Stored in directory: /root/.cache/pip/wheels/2a/85/33/2f6da85d5f10614cbe5a625eab3b3aebfdf43e7b857f25f829
Successfully built pyvi tabulate


In [0]:
from pyvi import ViTokenizer
from keras.preprocessing.sequence import pad_sequences
import string

def clean_document(doc):
    doc = ViTokenizer.tokenize(doc)
    doc = doc.lower() #Lower
    
    tokens = doc.split() #Split in_to words
    
    table = str.maketrans('', '', string.punctuation.replace("_", "")) #Remove all punctuation
    tokens = [w.translate(table) for w in tokens]
    
    tokens = [word for word in tokens if word]
        
    return tokens

In [0]:
import numpy as np

def preprocess_input(doc):
    tokens = clean_document(doc)
    tokens = tokenizer.texts_to_sequences(tokens)
    tokens = pad_sequences([tokens], maxlen=50, truncating='pre')
    return np.reshape(tokens, (1,50))

In [0]:
def generate_text(text_input, n_words):
    tokens = preprocess_input(text_input)
    for _ in range(n_words):
        next_digit = model.predict_classes(tokens)
        tokens = np.append(tokens, next_digit)
        tokens = np.delete(tokens, 0)
        tokens = np.reshape(tokens, (1, 50))
    
    # Mapping to text  
    tokens = np.reshape(tokens, (50))
    out_word = []
    for token in tokens:
        for word, index in tokenizer.word_index.items():
            if index == token:
                out_word.append(word)
                break

    return ' '.join(out_word)
            

In [40]:
seq = 'đường phố ở việt nam'
generate_text(text_input=seq, n_words=25)

'đường_phố ở việt_nam tạo nhiều nét của các nghệ_nhân chính_trị khác nhiều tác_phẩm được coi là nơi có nhiều cột đồ thu_hút được được đánh_giá là có giá_trị'

In [52]:
seq = 'Tại buổi đối thoại, nhiều doanh nghiệp cho biết hằng năm họ phải tiếp nhiều đoàn thanh tra, kiểm tra từ các cấp, việc này vô tình gây mất thời gian, phiền hà'
generate_text(text_input=seq, n_words=25)

'buổi đối_thoại nhiều doanh_nghiệp cho biết hằng năm họ phải tiếp nhiều đoàn thanh_tra kiểm_tra từ các cấp việc này vô_tình gây mất thời_gian phiền_hà lớn hơn cả năm nay các nhà đầu_tư lại thất_nghiệp vừa minh_bạch 2 usd trong khi nhà_nước có_thể xuất mức lãi_suất đang được tiếp_tục tiếp_tục'